# Creando modelos con TensorFlow y PyTorch

In [8]:
import torch
import deepchem as dc
import tensorflow as tf

En los tutoriales previos, hemos usado modelos estandar de DeepChem. Sin embargo, a veces querremos crear modelos propios con una arquitectura que nosotros definamos. DeepChem proporciona integración con TF y PT

Tenemos dos enfoques diferentes para utilizar modelos de TensorFlow o PyTorch con DeepChem. Estos enfoques dependen de si deseas utilizar las API de TensorFlow/PyTorch o las API de DeepChem para entrenar y evaluar tu modelo.

**Enfoque 1: Uso de TensorFlow o PyTorch con Dataset de DeepChem**

En este enfoque, puedes utilizar los datasets de DeepChem con TensorFlow o PyTorch. Puedes adaptar fácilmente los datasets de DeepChem para trabajar con otros frameworks. Aquí hay dos métodos para hacerlo:

1. `make_tf_dataset()`: Este método devuelve un objeto `tensorflow.data.Dataset` que itera sobre los datos.

2. `make_pytorch_dataset()`: Este método devuelve un objeto `torch.utils.data.IterableDataset` que itera sobre los datos. 

Esto te permite utilizar los datasets, cargadores, generadores de características, transformadores, separadores, etc. de DeepChem y fácilmente integrarlos en tu código existente de TensorFlow o PyTorch.

**Enfoque 2: Envolver tu modelo en un objeto DeepChem Model**

Si deseas aprovechar las características adicionales proporcionadas por DeepChem, puedes envolver tu modelo en un objeto Model de DeepChem. Esto te permite utilizar las funcionalidades adicionales que ofrece DeepChem, como la gestión de datasets y evaluación de modelos.


## KerasModel

KerasModel es una subclase de los modelos de DeepChem. Actua como un wrapper en torno a tensorflow.keras.Model.

In [3]:
import deepchem as dc
import tensorflow as tf

keras_model = tf.keras.Sequential([
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(1)
])
model = dc.models.KerasModel(keras_model, dc.models.losses.L2Loss())

Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (C:\Users\alloh\Miniconda3\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


1. **Capa `Dense` con 1000 neuronas y activación ReLU**:
   - La capa `Dense` es completamente conectada y cada unidad está vinculada a todas las unidades de la capa anterior.
   - `1000` es el número de neuronas en esta capa.
   - `activation='relu'` indica que se utiliza la función de activación ReLU (Rectified Linear Unit) en cada neurona. ReLU es una función no lineal comúnmente usada en redes neuronales.

2. **`Dropout` con una tasa del 50%**:
   - `Dropout` es una técnica de regularización que desactiva aleatoriamente un porcentaje de neuronas durante el entrenamiento para prevenir el sobreajuste. En este caso, se desactiva el 50% de las neuronas en cada paso.

3. **Capa `Dense` con una sola neurona**:
   - Esta capa `Dense` tiene una sola neurona y se utiliza para producir una salida escalar, típicamente en problemas de regresión donde se predice un valor numérico continuo en lugar de una clasificación.

En resumen, el modelo consta de dos capas densas con una capa de dropout entre ellas. La primera capa tiene 1000 neuronas con activación ReLU, mientras que la segunda capa tiene una sola neurona para generar la salida del modelo. El dropout se utiliza para regularizar el modelo y prevenir el sobreajuste, y esta arquitectura es común en tareas de regresión.


Para este ejemplo, utilizamos la clase `Sequential` de Keras. Nuestro modelo consiste en una capa densa con activación ReLU, un dropout del 50% para proporcionar regularización y una capa final que produce una salida escalar. También debemos especificar la función de pérdida que se utilizará al entrenar el modelo, en este caso, la pérdida L2. 

Ahora podemos entrenar y evaluar el modelo de la misma manera que lo haríamos con cualquier otro modelo de DeepChem. Por ejemplo, carguemos el conjunto de datos de solubilidad de Delaney. ¿Cómo se desempeña nuestro modelo en la predicción de las solubilidades de las moléculas basadas en sus Fingerprints (ECFPs)?


In [2]:
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='ECFP', splitter='random')
train_dataset, valid_dataset, test_dataset = datasets
model.fit(train_dataset, nb_epoch=50)
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print('training set score:', model.evaluate(train_dataset, [metric]))
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'pearson_r2_score': 0.9756559780179717}
test set score: {'pearson_r2_score': 0.7761586627436254}


## TorchModel

TorchModel funciona igual que KeralModel pero envolviendo al módulo torch.nn.Module. Usemos PyTorch para crear oro modelo como el anterior entrenando los mismos datos

In [3]:
import torch

pytorch_model = torch.nn.Sequential(
    torch.nn.Linear(1024, 1000),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(1000, 1)
)
model = dc.models.TorchModel(pytorch_model, dc.models.losses.L2Loss())

model.fit(train_dataset, nb_epoch=50)
print('training set score:', model.evaluate(train_dataset, [metric]))
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'pearson_r2_score': 0.9761305044359211}
test set score: {'pearson_r2_score': 0.776351241682339}


#### Explicación de `dc.models.losses.L2Loss()`

En DeepChem, `dc.models.losses.L2Loss()` es una función de pérdida utilizada para problemas de regresión. La pérdida L2, también conocida como pérdida de error cuadrado medio (MSE, por sus siglas en inglés, Mean Squared Error), mide la discrepancia entre las predicciones del modelo y los valores reales de las etiquetas mediante la diferencia al cuadrado. La fórmula para la pérdida L2 es la siguiente:

L2Loss = Σ(y_pred - y_actual)^2

Donde:
- `y_pred` son las predicciones del modelo.
- `y_actual` son los valores reales de las etiquetas.
- Σ denota la suma sobre todos los ejemplos de entrenamiento.

El objetivo de minimizar la pérdida L2 es hacer que las predicciones del modelo se acerquen lo más posible a los valores reales de las etiquetas, reduciendo así el error cuadrado medio.

Al utilizar `dc.models.losses.L2Loss()` en el modelo que has definido, estás configurando el modelo para que minimice la pérdida L2 durante el proceso de entrenamiento, lo que es adecuado para problemas de regresión en los que se busca predecir valores numéricos continuos.

Espero que esta explicación te sea útil.


## Computing Losses

En los ejemplos de arriba, el error (loss) se realizaba directamente desde la salida del modelo. A veces eso esta bien, pero no siempre. Por ejemplo, si tenemos un modelo de clasificación que nos devuelve una distribución de probabilidad, es mejor computar el error a traves de los logits[1]

***NOTA: Los logits son valores numéricos que representan las salidas sin procesar o crudas de un modelo de aprendizaje automático antes de aplicar una función de activación.***

Para hacer esto, crearemos un modelo que devuelva las dos salidas, las probabilidades y los logits. KerasModel y TorchModel te permite especificar una lista de outputs. Si un output es de tipo prediction significa que es una salida normal que debería devolver cuando llames a predict. Si tiene tipo 'loss', significa que debería pasarse a la función 'loss' en lugar de las salidas normales

Los modelos secuenciales no permiten multiples salidas, por lo que usaremos un modelo estilo subclase

In [6]:
class ClassificationModel(tf.keras.Model): #Subclase de tf.keras.Model
    
    def __init__(self):
        #Llamamos al constructor de la clase tf.keras.Model
        super(ClassificationModel, self).__init__()
        #Primera capa densa: 1000 neuronas, función reLU
        self.dense1 = tf.keras.layers.Dense(1000, activation='relu')
        #Segunda capa (salida): 1 neurona
        self.dense2 = tf.keras.layers.Dense(1)
    
        #La función call nos permitirá indicarle al modelo como se
        #infieren nuestros datos de entrada.
    def call(self, inputs, training=False):
        y = self.dense1(inputs) #Nuestros datos de entrada se pasan a la capa 1
        if training:
            y = tf.nn.dropout(y, 0.5) #Si estamos entrenando al modelo
                                      #usaremos un dropout de 0.5
        logits = self.dense2(y) #Pasamos las salidas de la primera capa
        #como entrada a la segunda y eso serán nuestros logits
        output = tf.nn.sigmoid(logits) #Usamos una función de activación
        #sigmoide para obtener la probabilidad a partir de los logits.
        return output, logits #Devolvemos tanto los outputs como los logits.

keras_model = ClassificationModel()

'''
Indicamos los tipos de salida. Si una salida tiene tipo 'prediction'
significará que es una salida normal que debería ser devuelta
cuando se llame a predict. Si tiene tipo 'loss', significará que
debería ser pasada a la función loss en vez de los outputs normales.
En nuestro caso, nuestra función loss es:
dc.models.losses.SigmoidCrossEntropy()
'''

output_types = ['prediction', 'loss']
model = dc.models.KerasModel(keras_model, dc.models.losses.SigmoidCrossEntropy(), output_types=output_types)

Lo que hacemos en el código de arriba es crear una clase llamada ClassificationModel que es una subclase de tf.keras.Model, es decir, que nuestra clase va a heredar las funcionalidades y comportamientos de la clase base tf.keras.Model.

Una vez hecho esto, definimos el constructor de nuestra clase. Primero llamamos al constructor de tf.keras.Model utilizando super() para asegurarnos de heredar todas las funcionalidades de tf.keras.Model y luego definimos dos capas densas, una de 1000 neuronas con activación ReLU y otra con una única neurona 

Lo siguiente que hacemos es definir la función Call. Este método se utiliza para definir como se va a realizar la inferencia de los datos del modelo. Es decir, cual va a ser el "viaje" de nuestros inputs hasta obtener su salida. En este caso, nuestros datos de entrada van a pasar por la primera capa de 1000 neuronas definidas en el constructor. Si estamos entrenando realizaremos un dropout de 0.5. Luego de que pasen por la primera capa, esos datos de salida de la primera capa pasarán a ser de entrada en la segunda capa (de una única neurona) y los datos de salida de esa neurona (antes de aplicar la función de activación) serán nuestros logits (que recordemos que era lo que nos interesa para el 'loss'). Luego ya por último definimos nuestros valores de salida mediante una función sigmoide y devolvemos tanto los outputs como los logits

Entrenamos el modelo usando BACE dataset. Es una clasificación binaria que predice si una molécula inhibirá a la encima BACE-1.

In [5]:
tasks, datasets, transformers = dc.molnet.load_bace_classification(feturizer='ECFP', splitter='scaffold')
train_dataset, valid_dataset, test_dataset = datasets
model.fit(train_dataset, nb_epoch=100)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('training set score:', model.evaluate(train_dataset, [metric]))
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'roc_auc_score': 0.9996367954180345}
test set score: {'roc_auc_score': 0.7727807971014493}


Para PyTorch es la misma filosofía solo que cambia un poco la sintaxis

In [9]:
class ClassificationModel(torch.nn.Module):
    
    def __init__(self):
        super(ClassificationModel, self).__init__()
        #1024 elementos de entrada // 1000 elementos de salida
        self.dense1 = torch.nn.Linear(1024, 1000)
        #1000 elementos de entrada // 1 elemento de salida
        self.dense2 = torch.nn.Linear(1000, 1)
        
    #Definición de la inferencia del modelo mediante forward.
    #Se especifica la lógica de procesamiento de datos de entrada
    def forward(self, inputs):
        y = torch.nn.functional.relu( self.dense1(inputs) )
        y = torch.nn.functional.dropout(y, p=0.5, training=self.training)
        logits = self.dense2(y)
        output = torch.sigmoid(logits)
        return output, logits

torch_model = ClassificationModel()
output_types = ['prediction', 'loss']
model = dc.models.TorchModel(torch_model, dc.models.losses.SigmoidCrossEntropy(), output_types=output_types)


In [10]:
tasks, datasets, transformers = dc.molnet.load_bace_classification(feturizer='ECFP', splitter='scaffold')
train_dataset, valid_dataset, test_dataset = datasets
model.fit(train_dataset, nb_epoch=100)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('training set score:', model.evaluate(train_dataset, [metric]))
print('test set score:', model.evaluate(test_dataset, [metric]))

training set score: {'roc_auc_score': 0.9996228260110358}
test set score: {'roc_auc_score': 0.7713315217391304}


## Otras Características

Tanto `KerasModel` como `TorchModel` tienen muchas otras características importantes. Aquí hay algunas de las más destacadas:

#### Guardado automático de puntos de control durante el entrenamiento.
- Los modelos `KerasModel` y `TorchModel` pueden guardar automáticamente puntos de control de tu modelo durante el entrenamiento. Esto permite que puedas recuperar modelos previos o continuar entrenando desde donde se quedó.

#### Registro del progreso en la consola, TensorBoard o Weights & Biases.
- Puedes registrar el progreso del entrenamiento en la consola, en TensorBoard o en la plataforma Weights & Biases. Esto te permite llevar un registro detallado del rendimiento de tu modelo durante el entrenamiento.

#### Funciones de pérdida personalizadas que puedes definir con una función de la forma f(outputs, labels, weights).
- Puedes definir tus propias funciones de pérdida personalizadas. Esto es útil cuando necesitas una función de pérdida específica para tu tarea de aprendizaje automático.

#### Detención temprana utilizando la clase ValidationCallback.
- Puedes implementar la detención temprana durante el entrenamiento utilizando la clase ValidationCallback. Esto te permite detener el entrenamiento si el rendimiento del modelo deja de mejorar en el conjunto de validación.

#### Cargar parámetros desde modelos pre-entrenados.
- Puedes cargar parámetros desde modelos pre-entrenados para iniciar tu modelo con pesos previamente aprendidos. Esto es útil en transferencia de aprendizaje.

#### Estimación de la incertidumbre en las salidas del modelo.
- Los modelos `KerasModel` y `TorchModel` te permiten estimar la incertidumbre en las salidas del modelo, lo que puede ser importante en aplicaciones de toma de decisiones críticas.

#### Identificación de características importantes mediante mapas de saliencia.
- Puedes utilizar mapas de saliencia para identificar las características importantes en tus datos y entender qué contribuye a las predicciones del modelo.

Al utilizar `KerasModel` o `TorchModel`, tienes acceso inmediato a todas estas características. Consulta la documentación de la API para obtener detalles completos sobre cómo utilizarlas.
